# QR Factorisation

We will look at two different ways for computing the QR factorisation of a matrix. To goal is to start from a matrix $A$ and write it as the product of an orthogonal matrix $Q$ and an upper-triangular matrix $R$.

In [1]:
import numpy as np
from numpy.linalg import norm

A = np.array([[1,2,3],[4,5,6],[7,8,10]])
b = np.array([1,2,3])
n = 3

Numpy has a built-in function for doing this. Let's use it to check what the answer should be:

In [2]:
A=np.array([[1,9,4],[0,4,3],[5,4,4]])
(Q,R) = np.linalg.qr(A)
(Q,R)

(array([[-0.19611614, -0.87794634, -0.43674785],
        [-0.        , -0.44539717,  0.8953331 ],
        [-0.98058068,  0.17558927,  0.08734957]]),
 array([[-5.09901951, -5.68736792, -4.70678724],
        [ 0.        , -8.98074864, -4.14561978],
        [ 0.        ,  0.        ,  1.28840617]]))

## Gram-Schmidt orthogonalization

The first approach will be to transform the vectors in the columns of $A$ to a set of orthogonal vectores using the Gram-Schmidt approach. The basic idea of Gram-Schmidt is to build up an orthonormal set of vectors by projecting out non-orthogonal pieces. The following image illustrates this.
![Gram-Schmidt Visualisation](Gram-Schmidt_orthonormalization_process.gif "Gram-Schmidt Visualisation")
Let's now implement this with our test matrix $A$.

First, we construct three vectors $a_1$, $a_2$ and $a_3$ from the columns of $A$.

In [3]:
(a1, a2, a3) = np.transpose(A)
a1


array([1, 0, 5])

Now, our first orthonormal vector is just $a_1$ normalised to have length 1:

In [4]:
u1 = a1
e1 = u1 / norm(u1)

To construct our second orthonormal vector, let's start with $a_2$, project out the part along the $a_1$ direction and normalise the result:

In [5]:
u2 = a2 - (e1@a2)*e1
e2 = u2/norm(u2)

Let's also project this piece out from $a_3$ now (this is not essential, but helps improve the numerical stability of the algorithm)

In [14]:
u3 = a3 - (e1@a3)*e1
u3

array([ 3.07692308,  3.        , -0.61538462])

To construct our third orthonormal vector, let's project out the part along the previous two directions and normalise the result:

In [15]:
u3 = u3 - (e2@a3)*e2
e3 = u3 / norm(u3)
[e1,e2,e3]
u3

array([-0.56270863,  1.15355269,  0.11254173])

Now we have our three orthogonal vectors, we can put them into the columns of Q

In [34]:
Q = np.transpose([e1, e2, e3])
Q

array([[ 0.19611614,  0.87794634, -0.43674785],
       [ 0.        ,  0.44539717,  0.8953331 ],
       [ 0.98058068, -0.17558927,  0.08734957]])

To get $R$, we note that $A = Q R$ means that $Q^T A = Q^T Q R = R$ since $Q$ is an orthogonal matrix. Let's use this to compute $R$:

In [16]:
R = np.transpose(Q)@A

In [19]:
np.transpose(Q)

array([[-0.19611614, -0.        , -0.98058068],
       [-0.87794634, -0.44539717,  0.17558927],
       [-0.43674785,  0.8953331 ,  0.08734957]])

In [20]:
Q@R

array([[ 1.00000000e+00,  9.00000000e+00,  4.00000000e+00],
       [-2.20160225e-17,  4.00000000e+00,  3.00000000e+00],
       [ 5.00000000e+00,  4.00000000e+00,  4.00000000e+00]])

As expected, $R$ is (almost) an upper-triangular matrix. It is only __almost__ upper triangular because floating point arithmetic is not exact.

## Using Householder reflections

The Gram-Schmidt process can be a very effective way to orthogonalise a set of vectors, but it does run into problems with numerical stability. This can happen, for example, in the case where we are starting with vectors that are nearly linearly dependent. Then we would be subtracting two large vectors to produce one small one, and we know that this is a recipe for disaster with floating point arithmetic.

One way around this problem is to use a different approach to orthogonalization. A very popular method uses the idea of Householder reflections. These take a vector $x$ and reflect it about a plane defined by another vector $v$:
![Householder reflection](Householder.png "Householder reflection")
This is clearly equivalent to multiplying $x$ by the __Householder reflection matrix__
$$ H = I - 2 \frac{v v^T}{||v||^2}$$
Note that $H$ is a *symmetric, orthogonal matrix*.

Now, if we choose $v$ appropriately then we can use it to zero out below the pivot in each column, thus producing $R$. In particular, if
$$v = a - ||a|| e_k$$ then $H a = |a| e_k$ so if $e_k$ is a unit vector in the $k$-th direction this does exactly what we want to the column $a$.

Let's now implement this in practice.

First, we work on the first column of $A$.

In [26]:
-np.sign(a3[0])
np.outer(v1,v1)
H1

array([[-0.19611614,  0.        , -0.98058068],
       [ 0.        ,  1.        ,  0.        ],
       [-0.98058068,  0.        ,  0.19611614]])

In [21]:
u1 = a1 - (-np.sign(a1[0]))*norm(a1)*np.array([1,0,0])
v1 = u1/norm(u1)
H1 = np.identity(3) - 2*np.outer(v1,v1)

In [35]:
A1 = H1@A
np.outer(v2,v2)

array([[ 0.72269858, -0.44766655],
       [-0.44766655,  0.27730142]])

and the second column

In [29]:
a2 = A1[1:,1]
u2 = a2 - (-np.sign(a2[0]))*norm(a2)*np.array([1,0])
v2 = u2/norm(u2)
H2 = np.identity(3)
H2[1:,1:] -= 2*np.outer(v2,v2)

In [63]:
H2

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        , -0.44539717,  0.8953331 ],
       [ 0.        ,  0.8953331 ,  0.44539717]])

In [40]:
A2 = H2@A1
A2
u3

array([2.57681234])

and finally the third column

In [43]:
a3 = A2[2:,2]
u3 = a3 - (-np.sign(a3[0]))*norm(a3)*np.array([1])
v3 = u3/norm(u3)
H3 = np.identity(3)
H3[2:,2:] -= 2*np.outer(v3,v3)

In [44]:
H3

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0., -1.]])

In [45]:
A3 = H3@A2

In [46]:
A3

array([[-5.09901951e+00, -5.68736792e+00, -4.70678724e+00],
       [-4.97009712e-16, -8.98074864e+00, -4.14561978e+00],
       [ 2.47245095e-16, -8.97768428e-16, -1.28840617e+00]])

We now have transformed to exactly $R$.

In [61]:
Q

array([[-0.19611614, -0.87794634,  0.43674785],
       [ 0.        , -0.44539717, -0.8953331 ],
       [-0.98058068,  0.17558927, -0.08734957]])

And we can easily construct $Q$ from the Householder matrices

In [60]:
Q=np.transpose(H3@H2@H1)

In [52]:
Q@R

array([[1.00000000e+00, 9.00000000e+00, 4.00000000e+00],
       [8.81388716e-32, 4.00000000e+00, 3.00000000e+00],
       [5.00000000e+00, 4.00000000e+00, 4.00000000e+00]])

In [41]:
Q@R

array([[ 1.00000000e+00,  9.00000000e+00,  4.00000000e+00],
       [-2.20160225e-17,  4.00000000e+00,  3.00000000e+00],
       [ 5.00000000e+00,  4.00000000e+00,  4.00000000e+00]])